In [1]:
 # -*- coding: utf-8 -*-
import pandas as pd

data = pd.read_csv('TB_RECIPE_SEARCH-20231130.csv')

In [2]:
# print(data.columns)
data.head()
data.shape

(184991, 18)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans

def preprocess_data(series):
    ingredient_list = []
    for data in series:
        if pd.notnull(data):  # NaN이 아닌 경우에만 처리
            # "|" 기호를 기준으로 문자열을 분리하여 각 재료와 해당 재료의 양을 추출
            ingredients = data.split("|")
            recipe_list = []
            for ingredient in ingredients:
                # "[재료]" 문자열 제거
                ingredient = ingredient.replace("[재료]", "").strip()
                parts = ingredient.split()
                if len(parts) >= 1:  # 재료 문자열이 올바른 형식인 경우에만 처리
                    ingredient_name = " ".join(parts[:-1])  # 재료 이름 추출
                    recipe_list.append(ingredient_name)
            if recipe_list:  # 재료가 있는 경우에만 추가
                ingredient_list.append(" | ".join(recipe_list))  # 리스트를 문자열로 변환하여 추가
    
    return ingredient_list


# 데이터 전처리
recipes = preprocess_data(data["CKG_MTRL_CN"])

# 샘플 수와 클러스터 개수 확인
num_samples = len(recipes)
num_clusters = 5  # 클러스터 개수
if num_samples >= num_clusters:
    # 피처 벡터화
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(recipes)

    # K-means 클러스터링 수행
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(X)

    # 클러스터링 결과 확인
    clustered_recipes = pd.DataFrame({"Recipe": recipes, "Cluster": kmeans.labels_})
    print(clustered_recipes)
else:
    print("Error: Number of samples is less than number of clusters.")

c:\Users\HYEONWOO\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


                                                   Recipe  Cluster
0       어묵 | 김밥용김 | 당면 | 양파 | 당근 | 깻잎 | 튀김가루 | 올리브유 | ...        2
1                        두부 | 당근 | 고추 | 브로콜리 | 새우 |  | 계란        2
2       밥 | 당근 | 치자단무지 | 신김치 | 무순 | 날치알 | 김가루 |  | 참기름...        2
3                                   현미 | 찹쌀 | 호두 | 물 | 소금        3
4       북어포 | 찹쌀가루 1C [양념] 간장 | 설탕 | 물 | 다진파 | 다진마늘 | ...        4
...                                                   ...      ...
183989            | 스파게티면 | 명란젓 | 생크림 | 마늘 | 양파 | 소금 | 후추        3
183990      | 스파게티면 | 관찰레 | 홀토마토 | 화이트와인 | 파르미지아노 치즈 | 소금        3
183991            |  |  |  |  | 쪽파 | 양파 | 다진마늘 | 통깨 | 참기름        4
183992  달걀 | 우유 | 물 | 맛살 | 당근 | 대파 1/3대 [양념] 소금 | 참치액 ...        3
183993                계란 | 물 동량 | 참치액 | 쪽파 | 알새우 |  |  |         2

[183994 rows x 2 columns]


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from konlpy.tag import Okt

# KoBERT 모델 로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base")

# Konlpy 형태소 분석기 초기화
okt = Okt()

# 뉴스 기사 본문 입력
text = """
    파이썬에는 NLP 관련 도구가 정말 많다. 원하는 텍스트 분석 작업에 적절한 파이썬 라이브러리는 어떻게 찾을 수 있을까? 용도별 추천 라이브러리와 장단점을 살펴본다. 

NLP(Natural Language Processing, 자연어 처리)는 ‘음성 및 텍스트를 위한 AI’라고 말할 수 있다. 음성 명령, 음성 및 텍스트 번역, 감성 분석, 텍스트 요약, 언어와 관련된 여타 많은 애플리케이션은 그 동안 크게 개선됐다. 분석에 사용되는 NLP의 성능 또한 딥러닝을 통해 획기적으로 향상됐다.

파이썬 언어는 NLP를 포함한 모든 종류의 머신러닝에 유용한 프론트엔드를 제공한다. 하지만 파이썬 생태계에는 고를 NLP가 너무 많아 당황스러운 것도 사실이다. 여기서는 주요 파이썬용 라이브러리의 사용례와 장단점 그리고 인기도를 살펴본다. 
 
CoreNLP
스탠포드 대학(Stanford University)이 만든 CoreNLP 라이브러리는 NLP 예측 및 분석 작업을 대규모로 수행할 수 있게 해주는 실용 단계의 NLP 솔루션이다. CoreNLP는 자바(Java)로 작성됐지만, 이를 위한 API와 여러 파이썬 패키지가 등장해 있는 상태다. Stanza로 불리는 네이티브 NLP 라이브러리가 그 중 하나다.

"""

# 형태소 분석 및 품사 태깅
nouns = []
for token in okt.pos(text):
    # 명사만 추출
    if token[1] in ["Noun", "Alpha"]:
        nouns.append(token[0])

# KoBERT 모델을 이용한 핵심 단어 추출
inputs = tokenizer(" ".join(nouns), return_tensors="pt")
outputs = model(**inputs)
predictions = outputs.logits.argmax(dim=-1)

# 단어 빈도 계산
word_counts = {}
for word in nouns:
    word_counts[word] = word_counts.get(word, 0) + 1

# 결과 출력
for word, count in sorted(word_counts.items(), key=lambda x: -x[1]):
    print(f"{word}: {count}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NLP: 8
파이썬: 6
라이브러리: 5
텍스트: 4
분석: 4
수: 3
음성: 3
및: 3
CoreNLP: 3
관련: 2
작업: 2
장단점: 2
위: 2
언어: 2
그: 2
러닝: 2
로: 2
도구: 1
정말: 1
용도: 1
별: 1
추천: 1
Natural: 1
Language: 1
Processing: 1
자연어: 1
처리: 1
AI: 1
말: 1
명령: 1
번역: 1
감성: 1
요약: 1
여타: 1
애플리케이션: 1
동안: 1
크게: 1
개선: 1
사용: 1
의: 1
성능: 1
또한: 1
딥: 1
통해: 1
획기: 1
향상: 1
를: 1
포함: 1
모든: 1
종류: 1
머신: 1
프론트엔드: 1
제공: 1
생태계: 1
당황: 1
것: 1
사실: 1
여기: 1
주요: 1
용: 1
용례: 1
인기: 1
스탠포드: 1
대학: 1
Stanford: 1
University: 1
이: 1
예측: 1
대규모: 1
수행: 1
실용: 1
단계: 1
솔루션: 1
자바: 1
Java: 1
작성: 1
API: 1
여러: 1
패키지: 1
등장: 1
상태: 1
Stanza: 1
네이티: 1
브: 1
중: 1
하나: 1


In [13]:
import torch
import torchvision
import torchaudio

print(f"PyTorch 버전: {torch.__version__}")
print(f"Torchvision 버전: {torchvision.__version__}")
print(f"Torchaudio 버전: {torchaudio.__version__}")

PyTorch 버전: 2.2.2+cpu
Torchvision 버전: 0.17.2+cpu
Torchaudio 버전: 2.2.2+cpu


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# KoBERT 모델 로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base")

# 토큰화 및 모델 입력 준비
inputs = []
for index, row in data.iterrows():
    text = " ".join(row.dropna())  # NaN 값 제거 후 단어들을 하나의 문자열로 결합
    inputs.append(tokenizer(text, return_tensors="pt"))

# 모델에 입력하고 예측 수행
outputs = []
for input_data in inputs:
    output = model(**input_data)
    outputs.append(output)

# 결과 출력
for output in outputs:
    predictions = output.logits.argmax(dim=-1)
    print(predictions)
